# GIFT数据处理

这一步是为了获得所有GIFT需要的数据


输入：
- GTEx等eQTL
- GTEx等对应的基因表达量矩阵
- GWAS结果
- pgen对应的LD

In [1]:
import pandas as pd 
from pathlib import Path
import numpy as np 
from io import StringIO
import pyranges as pr

# snpid tools
from finemap_tools.utils import add_ID, sort_ID
# load eQTLs function
from finemap_tools.reader.eQTL import  GTEx_tabix_reader


In [2]:
# topLoci = (1, 15583355, 15583355) # (chr, start, end)
# locus_range = 500 # 100kb 
# locus_range = locus_range * 1000 
# locus_range_tuple = (topLoci[0], topLoci[1] - locus_range, topLoci[2] + locus_range)




In [3]:
from typing import Tuple, Dict 
# 取顺式区域

def is_col_map_in_df(df:pd.DataFrame, col_map:Dict):

    not_in = []
    assert isinstance(col_map, dict), f"col_map should be a dict {col_map}"
    for _, v in col_map.items():
        if v not in df.columns.tolist():
            not_in.append(v)
    if len(not_in) == 0:
        return True
    else:
        print(f"{not_in } not in {df.columns.tolist()}")
        return False 


def detect_in_region(value, region_list):
    """
    Check if a value falls within any of the regions in the given region list.

    Parameters:
    value (float): The value to be checked.
    region_list (list): A list of tuples representing the regions. Each tuple should contain two elements: the lower bound and the upper bound of the region.

    Returns:
    bool: True if the value falls within any of the regions, False otherwise.
    """
    if isinstance(region_list[0], int):
        region_list = [region_list]

    for region in region_list:
        if all((value >= region[0], value < region[1])):

            return True 

    return False


def extract_cis_snps(df, pos_col,one_based_start_end):
    selector = df[pos_col].apply(lambda x: detect_in_region(x, one_based_start_end))
    return df[selector]

def extract_cis_snp_by_anno(eQTL_df:pd.DataFrame, genes_in_locus_df:pd.DataFrame, cis_range:int, eQTL_col_map:Dict, anno_file_col_map:Dict) -> pd.DataFrame:
    """
    Input:
    - eQTL_df: with chr pos 

    Output:
    - eQTL_df: with only cis-snp of any gene and have two new columns: gene_name, gene_id, which matched to the value in anno_file_col_map
    
    """
    eQTL_chr_col = eQTL_col_map['chr']
    eQTL_postion_col = eQTL_col_map['pos']
    eQTL_ref_col = eQTL_col_map['ref']
    eQTL_alt_col = eQTL_col_map['alt']

    eQTL_gene_id = eQTL_col_map['gene_id']

    anno_gene_id = anno_file_col_map['gene_id']
    anno_gene_name = anno_file_col_map['gene_name']
    anno_Start = anno_file_col_map['Start']
    anno_End = anno_file_col_map['End']

    eQTL_df['ID_sorted'] = add_ID(eQTL_df, [eQTL_chr_col, eQTL_postion_col, eQTL_ref_col, eQTL_alt_col], sort=True, inplace=False)

    # 1. eQTLs的基因必须在transcirpt list 
    # 2. eQTLs必须是cis-SNP

    ## 1. eQTLs的基因必须在transcirpt list

    print(f"the genes (from gene annotation file) in the locus: {genes_in_locus_df.shape[0]}")

    gene_in_locus_df_id_Name = genes_in_locus_df[[anno_gene_id, anno_gene_name]]

    matched = eQTL_df[eQTL_df[eQTL_gene_id].isin(gene_in_locus_df_id_Name[anno_gene_id])].merge(gene_in_locus_df_id_Name, left_on=eQTL_gene_id, right_on=anno_gene_id, how='inner')

    print(f"matched: {matched[eQTL_gene_id].unique().shape}")

    not_in_eQTL_gene_name = set(gene_in_locus_df_id_Name[anno_gene_name].tolist()) - set(matched[anno_gene_name].unique().tolist())

    print(f"not_in_eQTL_gene: {not_in_eQTL_gene_name}")
    gene_in_analysis = matched[anno_gene_name].unique().tolist()
    print(f"gene_in_analysis: {gene_in_analysis}")
    eQTL_df = matched

    genes_in_locus_df = genes_in_locus_df[genes_in_locus_df['Name'].isin(gene_in_analysis)] # update genes_in_locus_df


    ## 2. check cis-snp 
    cis_snp_list = []
    for geneName , gene_eQTLs in eQTL_df.groupby(anno_gene_name): 
        gene_region = genes_in_locus_df[genes_in_locus_df[anno_gene_name] == geneName].iloc[0][[anno_Start, anno_End]].tolist()
        upstream = (gene_region[0] - cis_range, gene_region[0])
        downstream = (gene_region[1], gene_region[1] + cis_range)
        gene_cis_range = [upstream, downstream]

        gene_cis_eQTLs_df = extract_cis_snps(gene_eQTLs, eQTL_postion_col, gene_cis_range)

        cis_snp_list += gene_cis_eQTLs_df['ID_sorted'].tolist()
        print(f"{geneName} have cis-snp num: {gene_cis_eQTLs_df.shape[0]} while total snp num: {gene_eQTLs.shape[0]} and gene_cis_range: {gene_cis_range}")


    cis_snp_list = set(cis_snp_list)
    print(f"total cis-snp num: {len(cis_snp_list)}")

    print("keep only it is cis-snp of any gene")
    print(f"before filter, eQTL_df shape: {eQTL_df.shape}")
    eQTL_df = eQTL_df[eQTL_df['ID_sorted'].isin(cis_snp_list)]
    print(f"after filter, eQTL_df shape: {eQTL_df.shape}")
    print(f"after filter, eQTL_df gene num: {eQTL_df['Name'].unique().shape}")
    print(f"after filter, eQTL_df snp num: {eQTL_df['ID_sorted'].unique().shape}")
    return eQTL_df

def GIFT_eQTL_pipline(eQTL_df, genes_in_locus_df, cis_range, eQTL_necessary_col_map, anno_file_necessary_col_map):
    ### cal TSTAT
    eQTL_df['TSTAT'] = eQTL_df[eQTL_necessary_col_map['beta']].astype(float) / eQTL_df[eQTL_necessary_col_map['se']].astype(float)
    print("have TSTAT na num is :",eQTL_df['TSTAT'].isna().sum())
    eQTL_df = eQTL_df.dropna(subset=['TSTAT'])
    print(f"after dropna: {eQTL_df.shape[0]}")

    ### extract cis-snp and two new columns: anno_file_necessary_col_map["gene_name"], anno_file_necessary_col_map["gene_id"] and "ID_sorted" in eQTL_df
    eQTL_df = extract_cis_snp_by_anno(eQTL_df = eQTL_df, genes_in_locus_df=genes_in_locus_df, cis_range=cis_range, eQTL_col_map=eQTL_necessary_col_map, anno_file_col_map=anno_file_necessary_col_map)

    ### gene in analysis 
    gene_in_analysis = eQTL_df[anno_file_necessary_col_map["gene_name"]].unique().tolist()

    ### remove any cis-snp not in all gene 
    gene_nums = len(gene_in_analysis)
    missing_cis_snp_rate = eQTL_df.groupby("ID_sorted").apply(lambda x: x.shape[0]/gene_nums).sort_values(ascending=False)
    missing_cis_snp = missing_cis_snp_rate[missing_cis_snp_rate == 0].index.tolist()
    print(f"missing rate >0: {len(missing_cis_snp)}")
    eQTL_df = eQTL_df[~eQTL_df['ID_sorted'].isin(missing_cis_snp)]
    print(f"eQTL_df: {eQTL_df.shape}")

    return eQTL_df, gene_in_analysis

default_eQTL_necessary_col = {
    'chr': 'chromosome',
    'pos': 'position',
    'ref': 'ref',
    'alt': 'alt',
    'beta': 'beta',
    'se': 'se',
    'gene_id': 'gene_id'
}
default_anno_file_necessary_col = {
    "gene_id": "gene_id",
    "gene_name": "Name",
    "Start": "Start",
    "End": "End"
}
default_gwas_necessary_col = {
    'chr': 'chrom',
    'pos': 'pos',
    'ref': 'ref',
    'alt': 'alt',
    'beta': 'beta',
    'se': 'sebeta'
}

def reorder_snplist_by_cis_snp_gene(
        eQTL_df:pd.DataFrame,
        genes_in_locus_df:pd.DataFrame,
        cis_range:int,
        eQTL_col_map:Dict,
        anno_file_col_map:Dict,
):
    snplist = [] 
    pindex = []
    gene = [] 
    eQTL_postion_col = eQTL_col_map['pos']
    eQTL_ID_col = "ID" # from pvar 
    anno_gene_name = anno_file_col_map['gene_name']
    anno_Start = anno_file_col_map['Start']
    anno_End = anno_file_col_map['End']

    for geneName, gene_eQTLs in eQTL_df.groupby(anno_gene_name):
        gene_region = genes_in_locus_df[genes_in_locus_df[anno_gene_name] == geneName].iloc[0][[anno_Start, anno_End]].tolist()
        upstream = (gene_region[0] - cis_range, gene_region[0])
        downstream = (gene_region[1], gene_region[1] + cis_range)
        gene_cis_range = [upstream, downstream]

        # filter eQTLs only keep the cis snps at cis range 
        gene_cis_eQTLs_df = extract_cis_snps(gene_eQTLs, eQTL_postion_col, gene_cis_range).set_index(eQTL_ID_col)[["TSTAT"]]
        if (na_num:=sum(gene_cis_eQTLs_df[['TSTAT']].isna().sum())) > 0:
            print(f"gene: {geneName} has {na_num} NA value in TSTAT")
            # gene_cis_eQTLs_df = gene_cis_eQTLs_df.dropna()
            raise Exception(f"gene: {geneName} has {na_num} NA value in TSTAT, plz check the data to ensure all cis-snp have TSTAT in eQTL data.")
            
        if gene_cis_eQTLs_df.shape[0] > 0 : # if there is any snp in the cis range
            # gene_eQTLs_cis[geneName] = gene_cis_eQTLs_df # TODO: append is not necessary 

            gene.append(geneName)
            snplist += gene_cis_eQTLs_df.index.tolist() # add cis snps to snplist
            pindex.append(gene_cis_eQTLs_df.shape[0]) # add the number of cis snps in the gene to pindex
        else:
            print(f"gene: {geneName} has no snp in cis range")



    print(f"total cis snps: {len(snplist)}")
    for snpnum, g in zip(pindex, gene):
        print(f"gene: {g} has {snpnum} cis snps")

    return snplist, pindex, gene

In [4]:
def ToGiFTFormatH5(eQTL_path, 
                   gwasPath,
                   locus_range_tuple,
                    ensemble_anno_hs_path,
                    pfile_path, 
                    gene_expression_path,

                   cis_range = 100, # kb 
                   gene_expression_format="GTEx",
                   eQTL_col_map=None,
                   gwas_col_map=None,
                   anno_file_col_map=None
                     ):
    """
    Input Format:
    1. eQTLpath: 应当是包含所有探针的eQTL文件，并且必须有对应的tabix索引；默认是对GTEx格式进行处理。如来源其他的地方，至少包含以下列：chr, pos, ref, alt ,beta, se, gene_id 7列，
    2. gwasPath: 应当是一个表型的GWAS文件，并且必须有tabix索引；默认是对GWASFormatted的格式进行处理，如果来源于其他的地方，至少包含以下列：chr, pos, ref, alt ,beta, se
    3. pvar_path 
    
    """
    # locus_range_update
    
    ## extract gene annotation from gff file 
    print("-------------------- loading anno file --------------------")
    try:
        ### load gff3
        ensemble_anno_hs = pr.read_gff3(ensemble_anno_hs_path) # load gff3
        ensemble_anno_hs_gene = ensemble_anno_hs[ensemble_anno_hs.Feature == 'gene'] 
        genes_in_locus_df = ensemble_anno_hs_gene[str(locus_range_tuple[0]), locus_range_tuple[1]:locus_range_tuple[2]].as_df()

        ### check col_map
        anno_file_necessary_col_map = default_anno_file_necessary_col
        if anno_file_col_map is not None:
            anno_file_necessary_col_map.update(anno_file_col_map)
        if not is_col_map_in_df(genes_in_locus_df, anno_file_necessary_col_map):
            raise ValueError(f"Error: anno_file_col_map is not valid: {anno_file_col_map}")

        ### step2 update new locus range with considering the boundary genes with cis-range 
        print(f"Considering the boundary of locus with cis range {cis_range//1000}kb with the old locus range: {locus_range_tuple}")
        locus_range_tuple_update = (locus_range_tuple[0], genes_in_locus_df['Start'].min() + cis_range, genes_in_locus_df['End'].max() + cis_range)
        print(f"new locus range: {locus_range_tuple_update}")

        ### locus_range_tuple_update
        locus_range_tuple = locus_range_tuple_update
    
    except:
        raise ValueError(f"Error: ensemble_anno_hs_path is not valid: {ensemble_anno_hs_path}")
    
    ## locus range
    chr = locus_range_tuple[0]
    start = locus_range_tuple[1]
    end = locus_range_tuple[2]
    region = f"{chr}:{start}-{end}"

    ## eQTL file parse 
    ### load eQTL file
    eQTL_df = GTEx_tabix_reader(eQTL_path, region)

    ### check col_map
    eQTL_necessary_col_map = default_eQTL_necessary_col
    if eQTL_col_map is not None:
        eQTL_necessary_col_map.update(eQTL_col_map)

    if not is_col_map_in_df(eQTL_df, eQTL_necessary_col_map):
        raise ValueError(f"Error: eQTL_col_map is not valid: {eQTL_col_map}")

    ### run GIFT eQTL pipline
    eQTL_df, gene_in_analysis = GIFT_eQTL_pipline(
        eQTL_df= eQTL_df,
        genes_in_locus_df=genes_in_locus_df,
        cis_range=cis_range,
        eQTL_necessary_col_map=eQTL_necessary_col_map,
        anno_file_necessary_col_map=anno_file_necessary_col_map
    )
    # ### cal TSTAT
    # eQTL_df['TSTAT'] = eQTL_df[eQTL_necessary_col_map['beta']].astype(float) / eQTL_df[eQTL_necessary_col_map['se']].astype(float)
    # print("have TSTAT na num is :",eQTL_df['TSTAT'].isna().sum())
    # eQTL_df = eQTL_df.dropna(subset=['TSTAT'])
    # print(f"after dropna: {eQTL_df.shape[0]}")

    # ### extract cis-snp and two new columns: anno_file_necessary_col_map["gene_name"], anno_file_necessary_col_map["gene_id"] and "ID_sorted" in eQTL_df
    # eQTL_df = extract_cis_snp_by_anno(eQTL_df = eQTL_df, genes_in_locus_df=genes_in_locus_df, cis_range=cis_range, eQTL_col_map=eQTL_necessary_col_map, anno_file_col_map=anno_file_necessary_col_map)

    # ### gene in analysis 
    # gene_in_analysis = eQTL_df[anno_file_necessary_col_map["gene_name"]].unique().tolist()

    # ### remove any cis-snp not in all gene 
    # gene_nums = len(gene_in_analysis)
    # missing_cis_snp_rate = eQTL_df.groupby("ID_sorted").apply(lambda x: x.shape[0]/gene_nums).sort_values(ascending=False)
    # missing_cis_snp = missing_cis_snp_rate[missing_cis_snp_rate == 0].index.tolist()
    # print(f"missing rate >0: {len(missing_cis_snp)}")
    # eQTL_df = eQTL_df[~eQTL_df['ID_sorted'].isin(missing_cis_snp)]
    # print(f"eQTL_df: {eQTL_df.shape}")


    ## gwas file parse 
    ### load gwas file
    from finemap_tools.reader.gwas import load_GWASFormated_file

    GWAS_df = load_GWASFormated_file(file_path=gwasPath, region=region)
    ### check col_map
    gwas_necessary_col_map = default_gwas_necessary_col
    if gwas_col_map is not None:
        gwas_necessary_col_map.update(gwas_col_map)

    if not is_col_map_in_df(GWAS_df, gwas_necessary_col_map):
        raise ValueError(f"Error: gwas_col_map is not valid: {gwas_col_map}")
    
    ### run GIFT GWAS pipline
    gwas_chr_col, gwas_pos_col, gwas_ref_col, gwas_alt_col, gwas_beta_col, gwas_se_col = gwas_necessary_col_map['chr'], gwas_necessary_col_map['pos'], gwas_necessary_col_map['ref'], gwas_necessary_col_map['alt'], gwas_necessary_col_map['beta'], gwas_necessary_col_map['se']

    GWAS_df['ID_sorted'] = add_ID(GWAS_df, [gwas_chr_col, gwas_pos_col, gwas_ref_col, gwas_alt_col], inplace=False, sort = True)
    GWAS_df["TSTAT"] = GWAS_df[gwas_beta_col].astype(float) / GWAS_df[gwas_se_col].astype(float)
    GWAS_df = GWAS_df.dropna(subset=["TSTAT"])

    ## check pfile for LD 

    from finemap_tools.reader.plink import read_pvar
    ### load pvar
    pvar_path = pfile_path + ".pvar"
    pvar_df = read_pvar(pvar_path)
    pvar_df['ID_sorted'] = sort_ID(pvar_df, "ID", inplace=False)

    ## intersection of eQTL and GWAS and pvar

    pvar_SNP = pvar_df['ID_sorted'].tolist()
    GWAS_SNP = GWAS_df['ID_sorted'].tolist()
    eQTL_SNP = eQTL_df['ID_sorted'].tolist()

    print(f"pvar_SNP: {len(pvar_SNP)}")
    print(f"GWAS_SNP: {len(GWAS_SNP)}")
    print(f"eQTL_SNP: {len(eQTL_SNP)}")

    pvar_SNP_set = set(pvar_SNP)
    GWAS_SNP_set = set(GWAS_SNP)
    eQTL_SNP_set = set(eQTL_SNP)

    intersection = pvar_SNP_set & GWAS_SNP_set & eQTL_SNP_set
    print(f"intersection: {len(intersection)}")

    pvar_df_intersction = pvar_df[pvar_df['ID_sorted'].isin(intersection)]
    # add real ID 
    GWAS_df_intersction = GWAS_df[GWAS_df['ID_sorted'].isin(intersection)].merge(pvar_df_intersction[['ID_sorted', "ID"]], on="ID_sorted")
    # add real ID 
    eQTL_df_intersction = eQTL_df[eQTL_df['ID_sorted'].isin(intersection)].merge(pvar_df_intersction[['ID_sorted', "ID"]], on="ID_sorted")

    SNP_ID_real = pvar_df_intersction["ID"].tolist()

    ## check afreq by plink2 and filter freq < 1e-2
    import tempfile
    from finemap_tools.plink import plink2_cal_freq

    with tempfile.TemporaryDirectory() as tmpdirname:
        print('created temporary directory', tmpdirname)
        freq = plink2_cal_freq(pgen=pfile_path, snplist=SNP_ID_real, outSuffix=tmpdirname + '/plink2_cal_freq')
        # print(freq)
    
    final_SNP_list = freq[freq['ALT_FREQS'] >1e-2]['ID'].tolist() 

    eQTL_df = eQTL_df_intersction[eQTL_df_intersction['ID'].isin(final_SNP_list)]
    GWAS_df = GWAS_df_intersction[GWAS_df_intersction['ID'].isin(final_SNP_list)]

    print(f"eQTL_df_passed_qc: {eQTL_df.shape}")
    print(f"GWAS_df_passed_qc: {GWAS_df.shape}")

    ### check any gene in eQTL_df has no cis-snp after QC
    gene_cis_snp_count = eQTL_df.groupby(anno_file_necessary_col_map["gene_name"]).apply(lambda x: x.shape[0])
    gene_cis_snp_count = gene_cis_snp_count[gene_cis_snp_count == 0]
    if gene_cis_snp_count.shape[0] > 0:
        # print(f"gene_cis_snp_count: {gene_cis_snp_count}")
        print(f"After QC theses genes have no cis-snp: {gene_cis_snp_count.index.tolist()}")
        eQTL_df = eQTL_df[~eQTL_df[anno_file_necessary_col_map["gene_name"]].isin(gene_cis_snp_count.index.tolist())]
    
    gene_in_analysis = eQTL_df[anno_file_necessary_col_map["gene_name"]].unique().tolist()

    ## cal gene expression df 

    ### load gene expression file

    from functools import reduce
    # snplist is all genes snp concat together, so may have duplicate snp
    def load_GTEx_gene_expression(geneExpression):
        geneExpression_df = pd.read_csv(geneExpression, sep = '\t', skiprows=2 ).iloc[:, 2:].set_index('Description').T
        return geneExpression_df
    geneExpression_df = load_GTEx_gene_expression(gene_expression_path)[gene_in_analysis]
    geneExpression_df_corr = geneExpression_df.corr()

    ## cal LD 
    from finemap_tools.plink import plink2_cal_LD
    with tempfile.TemporaryDirectory() as tmpdirname:
        print('created temporary directory', tmpdirname)
        LD_df = plink2_cal_LD(pgen=pfile_path, snplist=final_SNP_list, outSuffix=tmpdirname + '/plink2_cal_ld')

    ##  raarange cis-snp order of eQTL_df, GWAS_df, LD 
    ### generate snplist order 
    snplist, pindex, gene = reorder_snplist_by_cis_snp_gene(
        eQTL_df=eQTL_df,
        genes_in_locus_df=genes_in_locus_df, # only provide Start End info
        cis_range=cis_range,
        eQTL_col_map=eQTL_necessary_col_map,
        anno_file_col_map=anno_file_necessary_col_map
    )

    ### reorder eQTL_df, GWAS_df, LD_df
    snplist_df = pd.DataFrame(snplist, columns=["ID"])

    #Zscore_GWAS 计算
    Zscore_GWAS_df = snplist_df.merge(GWAS_df, how="left", on = "ID")[[ "ID", "TSTAT"]].set_index("ID")
    print(f"Zscore_GWAS_df: {Zscore_GWAS_df.shape}")
    # Zscore_eQTLs 计算
    Zscore_eQTLs_list = []

    for geneName, gene_eQTLs in eQTL_df.groupby(anno_file_necessary_col_map["gene_name"]):

        Zscore_eQTLs = snplist_df.merge(gene_eQTLs[[ "ID", "TSTAT"]].rename(columns = {"TSTAT": geneName}), how="left", on = "ID").set_index("ID")
        Zscore_eQTLs_list.append(Zscore_eQTLs)


    assert True == all((x.index == Zscore_eQTLs_list[0].index ).all() for x in Zscore_eQTLs_list)  # check if all index are the same
    Zscore_eQTLs_df = pd.concat(Zscore_eQTLs_list, axis = 1)
    print(f"Zscore_eQTLs_df: {Zscore_eQTLs_df.shape}")
    assert Zscore_eQTLs_df.isna().sum().sum() == 0, "eQTLs Zscore has NA value, plz check the data to ensure all cis-snp have TSTAT in eQTL data."

    # LD 重排顺序
    LD_eQTL = LD_df.loc[snplist, snplist]
    LD_GWAS = LD_df.loc[snplist, snplist]

    # expression corr 
    expression_corr = geneExpression_df_corr.loc[gene, gene]

    # check all 
    print("check all at final part~")
    print(Zscore_GWAS_df.shape[0], sum(pindex))
    assert sum(pindex) == Zscore_GWAS_df.shape[0], print(f"eQTLs snps are not equal to GWAS snps, eQTLs snps are {sum(pindex)} and GWAS snps are {Zscore_GWAS_df.shape[0]}")
    assert [col for col in Zscore_eQTLs_df.columns] == gene, print(f"gene is {gene}, while Zscore_eQTLs columns is {Zscore_eQTLs_df.columns.tolist()}")
    assert Zscore_GWAS_df.index.tolist() == Zscore_eQTLs_df.index.tolist()
    assert Zscore_GWAS_df.index.tolist() == snplist
    assert Zscore_eQTLs_df.index.tolist() == LD_eQTL.index.tolist()
    assert Zscore_GWAS_df.index.tolist() == LD_GWAS.index.tolist()
    assert expression_corr.index.tolist() == gene

    return gene, snplist, pindex, Zscore_eQTLs_df, Zscore_GWAS_df, LD_eQTL, LD_GWAS, expression_corr


In [7]:

# cis-snp range 
cis_range = 10 # 100kb 
print(f"cis_range: {cis_range}kb")
cis_range = cis_range * 1000

# locus range 
locus_range_tuple = (1, 15339446, 15817866)
base_dir = "/home/xutingfeng/GIFT/"
# necessary file path 
## ensemble annotation
ensemble_anno_hs_path = base_dir + "hs_anno/refseq/Homo_sapiens.GRCh38.111.gff3.gz" # refseq annotation

## GTEx tissue eQTL file
eQTLfile_path = base_dir+ "GTEx_V8/ge/Kidney_Cortex.tsv.gz"

## gwas file path 
gwasPath = base_dir +  "data/gwas/T1Mapping_Cortex_20240129.csv_firstorder_Median_all_2023_GRCh38_unionKidneys.tsv.gz"

## pvar path 
pfile_path = base_dir + "data/pgen/DNAJC16_GRCh38_union"

## gene expression

gene_expression_path = base_dir + "data/expression/gene_reads_2017-06-05_v8_kidney_cortex.gct.gz"

cis_range: 10kb


In [8]:
gene, snplist, pindex, Zscore_eQTLs_df, Zscore_GWAS_df, LD_eQTL, LD_GWAS, expression_corr = ToGiFTFormatH5(
    eQTL_path= eQTLfile_path,
    gwasPath= gwasPath,
    ensemble_anno_hs_path=ensemble_anno_hs_path,
    pfile_path= pfile_path,
    gene_expression_path=gene_expression_path,
    cis_range = cis_range,
    locus_range_tuple = locus_range_tuple

)




-------------------- loading anno file --------------------


/home/xutingfeng/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/xutingfeng/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:160: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  empty_removed = df.groupby(["Chromosome", "Strand"])


Considering the boundary of locus with cis range 10kb with the old locus range: (1, 15339446, 15817866)
new locus range: (1, 15246520, 15819348)
have TSTAT na num is : 3090
after dropna: 103680


/tmp/ipykernel_3637/2822344048.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eQTL_df['ID_sorted'] = add_ID(eQTL_df, [eQTL_chr_col, eQTL_postion_col, eQTL_ref_col, eQTL_alt_col], sort=True, inplace=False)
/tmp/ipykernel_3637/2822344048.py:130: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_cis_snp_rate = eQTL_df.groupby("ID_sorted").apply(lambda x: x.shape[0]/gene_nums).sort_values(ascending=False)


the genes (from gene annotation file) in the locus: 15
matched: (14,)
not_in_eQTL_gene: {'RSC1A1'}
gene_in_analysis: ['CELA2B', 'AGMAT', 'EFHD2', 'DNAJC16', 'SLC25A34', 'FHAD1', 'UQCRHL', 'DDI2', 'FBLIM1', 'CTRC', 'CELA2A', 'TMEM82', 'PLEKHM2', 'CASP9']
AGMAT have cis-snp num: 120 while total snp num: 2210 and gene_cis_range: [(15561698, 15571698), (15585051, 15595051)]
CASP9 have cis-snp num: 67 while total snp num: 2210 and gene_cis_range: [(15480831, 15490831), (15526534, 15536534)]
CELA2A have cis-snp num: 58 while total snp num: 2210 and gene_cis_range: [(15446727, 15456727), (15472091, 15482091)]
CELA2B have cis-snp num: 93 while total snp num: 2210 and gene_cis_range: [(15455908, 15465908), (15491395, 15501395)]
CTRC have cis-snp num: 65 while total snp num: 2210 and gene_cis_range: [(15428441, 15438441), (15449242, 15459242)]
DDI2 have cis-snp num: 59 while total snp num: 2210 and gene_cis_range: [(15607457, 15617457), (15669044, 15679044)]
DNAJC16 have cis-snp num: 134 while t

/tmp/ipykernel_3637/1729080893.py:167: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gene_cis_snp_count = eQTL_df.groupby(anno_file_necessary_col_map["gene_name"]).apply(lambda x: x.shape[0])


created temporary directory /tmp/tmpfn1_w_f7
total cis snps: 924
gene: AGMAT has 101 cis snps
gene: CASP9 has 65 cis snps
gene: CELA2A has 52 cis snps
gene: CELA2B has 86 cis snps
gene: CTRC has 57 cis snps
gene: DDI2 has 52 cis snps
gene: DNAJC16 has 115 cis snps
gene: EFHD2 has 58 cis snps
gene: FBLIM1 has 60 cis snps
gene: FHAD1 has 34 cis snps
gene: PLEKHM2 has 48 cis snps
gene: SLC25A34 has 48 cis snps
gene: TMEM82 has 45 cis snps
gene: UQCRHL has 103 cis snps
Zscore_GWAS_df: (924, 1)
Zscore_eQTLs_df: (924, 14)
check all at final part~
924 924


In [9]:
# save to h5 
import h5py

with h5py.File(base_dir+"GIFT.h5", "w") as f:
    f.create_group("data")
    f['data'].create_dataset('gene', data=gene)
    f['data'].create_dataset('snplist', data=snplist)
    f['data'].create_dataset('pindex', data=pindex)
    f['data'].create_dataset('Zscore_eQTLs_df', data=Zscore_eQTLs_df)
    f['data'].create_dataset('Zscore_GWAS_df', data=Zscore_GWAS_df)
    f['data'].create_dataset('geneExpression_corr_df', data=expression_corr)
    f['data'].create_dataset('LD_eQTL', data=LD_eQTL)
    f['data'].create_dataset('LD_GWAS', data=LD_GWAS)


In [60]:
from finemap_tools.reader.plink import read_pvar

pvar = read_pvar("data/pgen/DNAJC16_ukb.pvar")
pvar

,#CHROM,POS,ID,REF,ALT
0,1,14587143,1:14587143:C:T,T,C
1,1,14587157,1:14587157:C:T,C,T
2,1,14587159,1:14587159:A:G,G,A
3,1,14587190,1:14587190:A:C,C,A
4,1,14587192,1:14587192:C:T,T,C
...,...,...,...,...,...
109352,1,16587077,1:16587077:A:G,A,G
109353,1,16587078,1:16587078:G:T,T,G
109354,1,16587079,1:16587079:G:T,T,G
109355,1,16587082,1:16587082:C:T,T,C


In [96]:
from typing import overload
@overload
def is_ambiguous_alleles(id:str, ref:str=None, alt:str=None):...
@overload
def is_ambiguous_alleles( ref:str, alt:str,id:str=None):...

def is_ambiguous_alleles(id:str=None, ref:str=None, alt:str=None):
    """
    check if ref and alt is ambiguous alleles
    """
    if id is not None:
        chr, pos ,a1, a2 = id.split(":")
        # alt = id.split(":")[3]
    elif ref is not None and alt is not None:
        a1 = ref
        a2 = alt
    else:
        raise ValueError("plz provide id or ref and alt")
    
    if ref == "A" and alt == "T":
        return True
    elif ref == "T" and alt == "A":
        return True
    elif ref == "C" and alt == "G":
        return True
    elif ref == "G" and alt == "C":
        return True
    else:
        return False

from typing import List
@overload 
def find_Biallelic_snp(snp_id_list:List[str], id_sep:str):...
@overload
def find_Biallelic_snp(snplist_df:pd.DataFrame, pos_col:str, id_col:str, chr_col:str):...
def find_Biallelic_snp(snp_id_list:List[str]=None,id_sep:str=None,snplist_df:pd.DataFrame=None, pos_col:str=None, id_col:str=None, chr_col:str=None):
    if snp_id_list is not None and id_sep is not None:
        snplist_df = pd.DataFrame([[i] + i.split(id_sep) for i in snp_id_list], columns=["ID","chr", "pos", "A1", "A2"])
        pos_col = "pos"
        id_col = "ID"
        chr_col = "chr"
    elif snplist_df is not None and pos_col is not None and id_col is not None:
        snplist_df = snplist_df 

    pos_is_Biallelic = snplist_df.groupby([chr_col, pos_col]).apply(lambda x:  reset_df if (reset_df := x.drop_duplicates(subset=[id_col])).shape[0] > 1 else None).reset_index(drop=True)


    if len(pos_is_Biallelic.shape) == 0:
        return None 
    else:
        return pos_is_Biallelic
    # return pos_is_Biallelic



In [94]:
find_Biallelic_snp(["1:1234:A:T", "1:1234:C:G", "2:12345:C:G"], ":")

/tmp/ipykernel_27093/2044317120.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pos_is_Biallelic = snplist_df.groupby([chr_col, pos_col]).apply(lambda x:  reset_df if (reset_df := x.drop_duplicates(subset=[id_col])).shape[0] > 1 else None).reset_index(drop=True)


,ID,chr,pos,A1,A2
0,1:1234:A:T,1,1234,A,T
1,1:1234:C:G,1,1234,C,G


In [99]:
find_Biallelic_snp(snplist_df=pvar,pos_col="POS", id_col="ID", chr_col="#CHROM")

/tmp/ipykernel_27093/3581233435.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pos_is_Biallelic = snplist_df.groupby([chr_col, pos_col]).apply(lambda x:  reset_df if (reset_df := x.drop_duplicates(subset=[id_col])).shape[0] > 1 else None).reset_index(drop=True)


,#CHROM,POS,ID,REF,ALT
0,1,14598639,1:14598639:A:C,C,A
1,1,14598639,1:14598639:C:T,C,T
2,1,14598640,1:14598640:C:G,G,C
3,1,14598640,1:14598640:G:T,G,T
4,1,14598663,1:14598663:A:G,G,A
...,...,...,...,...,...
10852,1,16586951,1:16586951:G:T,G,T
10853,1,16586959,1:16586959:C:G,G,C
10854,1,16586959,1:16586959:G:T,G,T
10855,1,16587021,1:16587021:A:G,G,A


In [74]:
pd.DataFrame( [i.split(":") for i in snplist], columns= ["chr", "pos", "ref", "alt"]).drop_duplicates()['pos'].value_counts().sort_values()

pos
15471702    1
15471822    1
15471938    1
15472580    1
15473003    1
           ..
15905314    1
15906139    1
15895123    1
15893979    1
15908595    1
Name: count, Length: 1726, dtype: int64

## 提取该locus全部的基因

In [4]:
# step1 extract locus gene list 
# extract gene annotation from gff file 
ensemble_anno_hs = pr.read_gff3(ensemble_anno_hs_path) # load gff3
ensemble_anno_hs_gene = ensemble_anno_hs[ensemble_anno_hs.Feature == 'gene'] # focus on gene 
genes_in_locus_df = ensemble_anno_hs_gene[str(locus_range_tuple[0]), locus_range_tuple[1]:locus_range_tuple[2]].as_df()

/home/xutingfeng/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/xutingfeng/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:160: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  empty_removed = df.groupby(["Chromosome", "Strand"])


### 根据转录本的情况更新locus的范围

这步是为了locus两端的基因可以考虑到cis的全部

In [5]:
# step2 update new locus range with considering the boundary genes with cis-range 
print(f"Considering the boundary of locus with cis range {cis_range//1000}kb with the old locus range: {locus_range_tuple}")
locus_range_tuple_update = (locus_range_tuple[0], genes_in_locus_df['Start'].min() + cis_range, genes_in_locus_df['End'].max() + cis_range)
print(f"new locus range: {locus_range_tuple_update}")
# locus_range_tuple_update
locus_range_tuple = locus_range_tuple_update

Considering the boundary of locus with cis range 100kb with the old locus range: (1, 15339446, 15817866)
new locus range: (1, 15336520, 15909348)


## 读取 eQTL数据(GTEx)

In [7]:
# set the position col var 

default_eQTL_necessary_col = {
    'chr': 'chromosome',
    'pos': 'position',
    'ref': 'ref',
    'alt': 'alt',
    'beta': 'beta',
    'se': 'se',
    'gene_id': 'gene_id'
}
eQTL_chr_col = default_eQTL_necessary_col['chr']
eQTL_postion_col = default_eQTL_necessary_col['pos']
eQTL_ref_col = default_eQTL_necessary_col['ref']
eQTL_alt_col = default_eQTL_necessary_col['alt']

eQTL_beta_col = default_eQTL_necessary_col['beta']
eQTL_se_col = default_eQTL_necessary_col['se']

eQTL_gene_id = default_eQTL_necessary_col['gene_id']


chr = locus_range_tuple[0]
start = locus_range_tuple[1]
end = locus_range_tuple[2]
region = f"{chr}:{start}-{end}"

eQTL_df = GTEx_tabix_reader(eQTLfile_path, region)

# set ID_sorted for intersection at later 
eQTL_df['ID_sorted'] = add_ID(eQTL_df, [eQTL_chr_col, eQTL_postion_col, eQTL_ref_col, eQTL_alt_col], sort=True, inplace=False)

# cal TSTAT for zscore matrix and drop na with TSTAT
eQTL_df['TSTAT'] = eQTL_df[eQTL_beta_col].astype(float) / eQTL_df[eQTL_se_col].astype(float)
print("have TSTAT na num is :",eQTL_df['TSTAT'].isna().sum())
eQTL_df = eQTL_df.dropna(subset=['TSTAT'])
print(f"after dropna: {eQTL_df.shape[0]}")


### eQTL和 transcript 取交集
# 1. eQTLs的基因必须在transcirpt list 
# 2. eQTLs必须是cis-SNP
# 3. 这个cis-SNP 必须所有gene都有对应的TSTAT

# 1. check gene in transcript gene list 
print(f"the genes (from gene annotation file) in the locus: {genes_in_locus_df.shape[0]}")

gene_in_locus_df_id_Name = genes_in_locus_df[['gene_id', 'Name']]

matched = eQTL_df[eQTL_df['gene_id'].isin(gene_in_locus_df_id_Name['gene_id'])].merge(gene_in_locus_df_id_Name, on="gene_id")

print(f"matched: {matched['gene_id'].unique().shape}")

not_in_eQTL_gene_name = set(gene_in_locus_df_id_Name['Name'].tolist()) - set(matched['Name'].unique().tolist())

print(f"not_in_eQTL_gene: {not_in_eQTL_gene_name}")
gene_in_analysis = matched['Name'].unique().tolist()
print(f"gene_in_analysis: {gene_in_analysis}")
eQTL_df = matched

genes_in_locus_df = genes_in_locus_df[genes_in_locus_df['Name'].isin(gene_in_analysis)] # update genes_in_locus_df

# 2. check cis-snp 

cis_snp_list = []

for geneName , gene_eQTLs in eQTL_df.groupby('Name'): 
    gene_region = genes_in_locus_df[genes_in_locus_df['Name'] == geneName].iloc[0][['Start', "End"]].tolist()
    upstream = (gene_region[0] - cis_range, gene_region[0])
    downstream = (gene_region[1], gene_region[1] + cis_range)
    gene_cis_range = [upstream, downstream]

    gene_cis_eQTLs_df = extract_cis_snps(gene_eQTLs, eQTL_postion_col, gene_cis_range)

    cis_snp_list = cis_snp_list + gene_cis_eQTLs_df['ID_sorted'].tolist()
    print(f"{geneName} have cis-snp num: {gene_cis_eQTLs_df.shape[0]} while total snp num: {gene_eQTLs.shape[0]}")


cis_snp_list = set(cis_snp_list)
print(f"total cis-snp num: {len(cis_snp_list)}")

print("keep only it is cis-snp of any gene")
print(f"before filter, eQTL_df shape: {eQTL_df.shape}")
eQTL_df = eQTL_df[eQTL_df['ID_sorted'].isin(cis_snp_list)]
print(f"after filter, eQTL_df shape: {eQTL_df.shape}")
print(f"after filter, eQTL_df gene num: {eQTL_df['Name'].unique().shape}")
print(f"after filter, eQTL_df snp num: {eQTL_df['ID_sorted'].unique().shape}")
# 3. 去除缺失率过高的SNP
gene_nums = len(gene_in_analysis)
missing_SNP_rate = eQTL_df.groupby("ID_sorted").apply(lambda x: x.shape[0]/gene_nums).sort_values(ascending=False)
# missing_SNP_rate 
missing_SNP = missing_SNP_rate[missing_SNP_rate == 0].index.tolist()
print(f"missing rate >0: {len(missing_SNP)}")

eQTL_df = eQTL_df[~eQTL_df['ID_sorted'].isin(missing_SNP)]

print(f"eQTL_df: {eQTL_df.shape}")



have TSTAT na num is : 5217
after dropna: 101141
the genes (from gene annotation file) in the locus: 15
matched: (14,)
not_in_eQTL_gene: {'RSC1A1'}
gene_in_analysis: ['CELA2B', 'TMEM82', 'CELA2A', 'CTRC', 'UQCRHL', 'DNAJC16', 'CASP9', 'FBLIM1', 'DDI2', 'SLC25A34', 'AGMAT', 'PLEKHM2', 'EFHD2', 'FHAD1']
AGMAT have cis-snp num: 719 while total snp num: 2058
CASP9 have cis-snp num: 808 while total snp num: 2058
CELA2A have cis-snp num: 731 while total snp num: 2058
CELA2B have cis-snp num: 798 while total snp num: 2058
CTRC have cis-snp num: 764 while total snp num: 2058
DDI2 have cis-snp num: 770 while total snp num: 2058
DNAJC16 have cis-snp num: 731 while total snp num: 2058
EFHD2 have cis-snp num: 653 while total snp num: 2058
FBLIM1 have cis-snp num: 704 while total snp num: 2058
FHAD1 have cis-snp num: 398 while total snp num: 2058
PLEKHM2 have cis-snp num: 769 while total snp num: 2058
SLC25A34 have cis-snp num: 711 while total snp num: 2058
TMEM82 have cis-snp num: 705 while total 

/tmp/ipykernel_968/3075796394.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_SNP_rate = eQTL_df.groupby("ID_sorted").apply(lambda x: x.shape[0]/gene_nums).sort_values(ascending=False)


## 读取GWAS数据 (GWASFormatted file)
<!-- regenie格式 -->

In [8]:
from finemap_tools.reader.gwas import load_GWASFormated_file

default_gwas_necessary_col = {
    'chr': 'chrom',
    'pos': 'pos',
    'ref': 'ref',
    'alt': 'alt',
    'beta': 'beta',
    'se': 'sebeta'
}
gwas_chr_col = default_gwas_necessary_col['chr']
gwas_pos_col = default_gwas_necessary_col['pos']
gwas_ref_col = default_gwas_necessary_col['ref']
gwas_alt_col = default_gwas_necessary_col['alt']

gwas_beta_col = default_gwas_necessary_col['beta']
gwas_se_col = default_gwas_necessary_col['se']


print(f"loading region: {region}")
GWAS_df = load_GWASFormated_file(file_path=gwasPath, region=region)

GWAS_df['ID_sorted'] = add_ID(GWAS_df, [gwas_chr_col, gwas_pos_col, gwas_ref_col, gwas_alt_col], inplace=False, sort = True)
GWAS_df["TSTAT"] = GWAS_df[gwas_beta_col].astype(float) / GWAS_df[gwas_se_col].astype(float)
GWAS_df = GWAS_df.dropna(subset=["TSTAT"])

loading region: 1:15336520-15909348
tabix have a header, so will take the first line as header and remove it.


## check pfile for LD 

In [21]:
from finemap_tools.reader.plink import read_pvar

pvar_df = read_pvar(pvar_path)

pvar_df['ID_sorted'] = sort_ID(pvar_df, "ID", inplace=False)

,#CHROM,POS,ID,REF,ALT
0,1,14587143,1:14587143:C:T,T,C
1,1,14587157,1:14587157:C:T,C,T
2,1,14587159,1:14587159:A:G,G,A
3,1,14587190,1:14587190:A:C,C,A
4,1,14587192,1:14587192:C:T,T,C
...,...,...,...,...,...
109352,1,16587077,1:16587077:A:G,A,G
109353,1,16587078,1:16587078:G:T,T,G
109354,1,16587079,1:16587079:G:T,T,G
109355,1,16587082,1:16587082:C:T,T,C


## 所有纳入的SNP进行QC


1. 交集
2. MAF > 1e-2

- 这里PGEN是UKB的数据，因此MAF以UKB的数据为准进行计算

- 也可以gwas的数据为准进行MAF过滤，不可以eQTL（因为summary数据大多样本量较小）

In [23]:
pvar_SNP = pvar_df['ID_sorted'].tolist()
GWAS_SNP = GWAS_df['ID_sorted'].tolist()
eQTL_SNP = eQTL_df['ID_sorted'].tolist()


print(f"pvar_SNP: {len(pvar_SNP)}")
print(f"GWAS_SNP: {len(GWAS_SNP)}")
print(f"eQTL_SNP: {len(eQTL_SNP)}")


pvar_SNP_set = set(pvar_SNP)
GWAS_SNP_set = set(GWAS_SNP)
eQTL_SNP_set = set(eQTL_SNP)


intersection = pvar_SNP_set & GWAS_SNP_set & eQTL_SNP_set
print(f"intersection: {len(intersection)}")

pvar_df_intersction = pvar_df[pvar_df['ID_sorted'].isin(intersection)]
# add real ID 
GWAS_df_intersction = GWAS_df[GWAS_df['ID_sorted'].isin(intersection)].merge(pvar_df_intersction[['ID_sorted', "ID"]], on="ID_sorted")
# add real ID 
eQTL_df_intersction = eQTL_df[eQTL_df['ID_sorted'].isin(intersection)].merge(pvar_df_intersction[['ID_sorted', "ID"]], on="ID_sorted")


SNP_ID_real = pvar_df_intersction["ID"].tolist()
# SNP_ID_real


pvar_SNP: 109357
GWAS_SNP: 4458
eQTL_SNP: 28812
intersection: 2027


In [24]:
import tempfile
from finemap_tools.plink import plink2_cal_freq


In [25]:
# 创建临时文件夹来存储数据
with tempfile.TemporaryDirectory() as tmpdirname:
    print('created temporary directory', tmpdirname)
    freq = plink2_cal_freq(pgen="data/pgen/DNAJC16_ukb", snplist=SNP_ID_real, outSuffix=tmpdirname + '/plink2_cal_freq')
    print(freq)
    
# freq = plink2_cal_freq(pgen="data/pgen/DNAJC16_ukb", snplist=SNP_ID_real, outSuffix='test')

created temporary directory /tmp/tmp80w217vi


      #CHROM                 ID REF   ALT PROVISIONAL_REF?  ALT_FREQS  OBS_CT
0          1     1:15336612:A:C   C     A                Y   0.013270  911472
1          1     1:15336888:A:G   A     G                Y   0.072879  932296
2          1     1:15336935:C:T   T     C                Y   0.306207  921016
3          1     1:15337191:A:G   G     A                Y   0.013222  931276
4          1     1:15337448:C:T   C     T                Y   0.065020  930034
...      ...                ...  ..   ...              ...        ...     ...
2022       1     1:15905314:C:G   C     G                Y   0.085668  932098
2023       1     1:15906139:C:T   C     T                Y   0.071549  932296
2024       1     1:15906557:C:T   C     T                Y   0.002449  932132
2025       1  1:15908170:G:GCTA   G  GCTA                Y   0.002481  932166
2026       1     1:15908595:A:G   A     G                Y   0.017554  929088

[2027 rows x 7 columns]


In [26]:
final_SNP_list = freq[freq['ALT_FREQS'] >1e-2]['ID'].tolist() 

eQTL_df = eQTL_df_intersction[eQTL_df_intersction['ID'].isin(final_SNP_list)]
GWAS_df_passed_qc = GWAS_df_intersction[GWAS_df_intersction['ID'].isin(final_SNP_list)]
print(f"eQTL_df_passed_qc: {eQTL_df.shape}")
print(f"GWAS_df_passed_qc: {GWAS_df_passed_qc.shape}")

eQTL_df_passed_qc: (24164, 22)
GWAS_df_passed_qc: (1726, 11)


## 基因表达量相关性计算


In [27]:

    
expression_filepath = "data/expression/gene_reads_2017-06-05_v8_kidney_cortex.gct.gz"



In [28]:
geneExpression_df = load_GTEx_gene_expression(expression_filepath)[gene_in_analysis]
geneExpression_df_corr = geneExpression_df.corr()
geneExpression_df_corr

Description,CELA2B,TMEM82,CELA2A,CTRC,UQCRHL,DNAJC16,CASP9,FBLIM1,DDI2,SLC25A34,AGMAT,PLEKHM2,EFHD2,FHAD1
Description,,,,,,,,,,,,,,
CELA2B,1.000000,-0.184775,0.942457,0.979194,-0.083674,-0.078751,0.165051,0.208723,0.038826,-0.048049,-0.111831,0.071931,0.062023,0.249947
TMEM82,-0.184775,1.000000,-0.202855,-0.180160,0.363203,0.553856,0.136347,-0.095800,0.418527,0.657048,0.377137,0.087738,0.050699,0.229548
CELA2A,0.942457,-0.202855,1.000000,0.947389,-0.069936,-0.090878,0.265227,0.135480,0.010767,-0.033191,-0.120507,0.031208,0.012756,0.200312
CTRC,0.979194,-0.180160,0.947389,1.000000,-0.058232,-0.053476,0.200043,0.199330,0.062109,-0.024640,-0.088197,0.071908,0.067095,0.243770
UQCRHL,-0.083674,0.363203,-0.069936,-0.058232,1.000000,0.583717,0.579547,0.042745,0.460671,0.389708,0.438251,0.359547,0.239764,0.227805
DNAJC16,-0.078751,0.553856,-0.090878,-0.053476,0.583717,1.000000,0.391077,-0.060071,0.475638,0.478915,0.891022,0.094745,0.094650,0.293590
CASP9,0.165051,0.136347,0.265227,0.200043,0.579547,0.391077,1.000000,0.176401,0.582608,0.338586,0.128561,0.491348,0.284614,0.460086
FBLIM1,0.208723,-0.095800,0.135480,0.199330,0.042745,-0.060071,0.176401,1.000000,0.517010,0.066621,-0.294383,0.766109,0.740864,0.356500
DDI2,0.038826,0.418527,0.010767,0.062109,0.460671,0.475638,0.582608,0.517010,1.000000,0.454545,0.139019,0.730252,0.586365,0.541152


## LD 数据计算

In [30]:
LD_df = plink2_cal_LD(pgen="data/pgen/DNAJC16_ukb", snplist=final_SNP_list, outSuffix='test')
LD_df

,1:15336612:A:C,1:15336888:A:G,1:15336935:C:T,1:15337191:A:G,1:15337448:C:T,1:15337531:C:T,1:15337851:A:C,1:15337914:C:T,1:15339446:G:T,1:15340368:A:G,...,1:15903102:A:T,1:15903567:A:G,1:15903632:T:TC,1:15904162:C:T,1:15904535:C:T,1:15904647:A:C,1:15905251:C:T,1:15905314:C:G,1:15906139:C:T,1:15908595:A:G
1:15336612:A:C,1.000000,0.000999,0.006069,0.000215,0.000782,0.000269,0.000780,0.000256,0.000180,0.000625,...,0.005622,0.005153,0.000126,0.004777,0.000066,0.000065,0.000242,0.000001,0.008599,0.000021
1:15336888:A:G,0.000999,1.000000,0.036063,0.000217,0.885309,0.002126,0.882178,0.001493,0.000233,0.742131,...,0.000334,0.001731,0.000460,0.001791,0.000046,0.000009,0.000623,0.001050,0.001502,0.000186
1:15336935:C:T,0.006069,0.036063,1.000000,0.006285,0.031516,0.026295,0.031567,0.007372,0.005547,0.026510,...,0.002184,0.001505,0.000388,0.001194,0.000615,0.000028,0.000916,0.003779,0.000306,0.003086
1:15337191:A:G,0.000215,0.000217,0.006285,1.000000,0.000729,0.000368,0.000746,0.000225,0.956575,0.000776,...,0.000008,0.000008,0.000007,0.000024,0.000069,0.000117,0.000418,0.000042,0.000010,0.000036
1:15337448:C:T,0.000782,0.885309,0.031516,0.000729,1.000000,0.001775,0.997336,0.001231,0.000670,0.857628,...,0.001038,0.001305,0.000171,0.001419,0.000005,0.000009,0.000008,0.000372,0.002889,0.000066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1:15904647:A:C,0.000065,0.000009,0.000028,0.000117,0.000009,0.000006,0.000010,0.000217,0.000107,0.000020,...,0.002028,0.002198,0.466997,0.002287,0.000225,1.000000,0.013380,0.001743,0.001306,0.000305
1:15905251:C:T,0.000242,0.000623,0.000916,0.000418,0.000008,0.001184,0.000019,0.000044,0.000526,0.000074,...,0.078008,0.084164,0.034112,0.089055,0.009350,0.013380,1.000000,0.071025,0.052120,0.012263
1:15905314:C:G,0.000001,0.001050,0.003779,0.000042,0.000372,0.002673,0.000354,0.000612,0.000087,0.000334,...,0.010825,0.010410,0.004692,0.011098,0.001258,0.001743,0.071025,1.000000,0.006990,0.001791
1:15906139:C:T,0.008599,0.001502,0.000306,0.000010,0.002889,0.000036,0.002890,0.000530,0.000003,0.003745,...,0.664067,0.639381,0.003635,0.605338,0.000914,0.001306,0.052120,0.006990,1.000000,0.001348


<!-- ## eQTLs 提取顺式区域

这一步会生成：
- snplist: (gene1_cis_snp, ...) 
- pindex: (gene1_cis_snp_num, gene2_cis_snp_num ...)
- Zscore_eQTLs (snplist_nums, gene_nums) -->

In [31]:
eQTL_postion_col = "position"
eQTL_ID_col = "ID"
snplist = [] 
pindex = []
gene = [] 
# gene_eQTLs_cis = {} 

print(f"cis range: {cis_range//1000}kb")



for geneName, gene_eQTLs in eQTL_df.groupby("Name"):
    gene_region = genes_in_locus_df[genes_in_locus_df["Name"] == geneName].iloc[0][['Start', "End"]].tolist()
    upstream = (gene_region[0] - cis_range, gene_region[0])
    downstream = (gene_region[1], gene_region[1] + cis_range)
    gene_cis_range = [upstream, downstream]

    # filter eQTLs only keep the cis snps at cis range 
    gene_cis_eQTLs_df = extract_cis_snps(gene_eQTLs, eQTL_postion_col, gene_cis_range).set_index(eQTL_ID_col)[["TSTAT"]]
    if (na_num:=sum(gene_cis_eQTLs_df[['TSTAT']].isna().sum())) > 0:
        print(f"gene: {geneName} has {na_num} NA value in TSTAT")
        # gene_cis_eQTLs_df = gene_cis_eQTLs_df.dropna()
        raise Exception(f"gene: {geneName} has {na_num} NA value in TSTAT, plz check the data to ensure all cis-snp have TSTAT in eQTL data.")
        
    if gene_cis_eQTLs_df.shape[0] > 0 : # if there is any snp in the cis range
        # gene_eQTLs_cis[geneName] = gene_cis_eQTLs_df # TODO: append is not necessary 

        gene.append(geneName)
        snplist += gene_cis_eQTLs_df.index.tolist() # add cis snps to snplist
        pindex.append(gene_cis_eQTLs_df.shape[0]) # add the number of cis snps in the gene to pindex
    else:
        print(f"gene: {geneName} has no snp in cis range")



print(f"total cis snps: {len(snplist)}")
for snpnum, g in zip(pindex, gene):
    print(f"gene: {g} has {snpnum} cis snps")


cis range: 100kb
total cis snps: 8430
gene: AGMAT has 647 cis snps
gene: CASP9 has 674 cis snps
gene: CELA2A has 618 cis snps
gene: CELA2B has 678 cis snps
gene: CTRC has 639 cis snps
gene: DDI2 has 656 cis snps
gene: DNAJC16 has 639 cis snps
gene: EFHD2 has 552 cis snps
gene: FBLIM1 has 583 cis snps
gene: FHAD1 has 324 cis snps
gene: PLEKHM2 has 650 cis snps
gene: SLC25A34 has 614 cis snps
gene: TMEM82 has 600 cis snps
gene: UQCRHL has 556 cis snps


## 整合数据=> GIFT格式
需要
- eQTL
- GWAS 
- LD 
- R  N by N  matrix, where N is gene nums
- gene list (gene1, gene2 ...)



重排LD、eQTL，GWAS，R以及genelist:

- eQTL: row is snplist , columns is genes 
- GWAS: row is snp list 
- LD: row and column is snplist 
- R: N by N matrix, N is gene nums 
- gene list (gene1, gene2 ...)



In [39]:
snplist_df = pd.DataFrame(snplist, columns=["ID"])

#Zscore_GWAS 计算
Zscore_GWAS_df = snplist_df.merge(GWAS_df_passed_qc, how="left", on = "ID")[[ "ID", "TSTAT"]].set_index("ID")

# Zscore_eQTLs 计算
Zscore_eQTLs_list = []
for geneName, gene_eQTLs in eQTL_df.groupby("Name"):

    Zscore_eQTLs = snplist_df.merge(gene_eQTLs[[ "ID", "TSTAT"]].rename(columns = {"TSTAT": geneName}), how="left", on = "ID").set_index("ID")
    Zscore_eQTLs_list.append(Zscore_eQTLs)

assert True == all((x.index == Zscore_eQTLs_list[0].index ).all() for x in Zscore_eQTLs_list)  # check if all index are the same
Zscore_eQTLs_df = pd.concat(Zscore_eQTLs_list, axis = 1)
assert Zscore_eQTLs_df.isna().sum().sum() == 0, "eQTLs Zscore has NA value, plz check the data to ensure all cis-snp have TSTAT in eQTL data."

# LD 重排顺序
LD_eQTL = LD_df.loc[snplist, snplist]
LD_GWAS = LD_df.loc[snplist, snplist]

# expression corr 
expression_corr = geneExpression_df_corr.loc[gene, gene]

# check all 
print(Zscore_GWAS_df.shape[0], sum(pindex))
assert sum(pindex) == Zscore_GWAS_df.shape[0], print(f"eQTLs snps are not equal to GWAS snps, eQTLs snps are {sum(pindex)} and GWAS snps are {Zscore_GWAS_df.shape[0]}")
assert [col for col in Zscore_eQTLs_df.columns] == gene
assert Zscore_GWAS_df.index.tolist() == Zscore_eQTLs_df.index.tolist()
assert Zscore_GWAS_df.index.tolist() == snplist
assert Zscore_eQTLs_df.index.tolist() == LD_eQTL.index.tolist()
assert Zscore_GWAS_df.index.tolist() == LD_GWAS.index.tolist()
assert expression_corr.index.tolist() == gene

print(f"eQTLs matrix shape is :{Zscore_eQTLs_df.shape}")




8430 8430
eQTLs matrix shape is :(8430, 14)


In [40]:
# save to h5 
import h5py

with h5py.File("GIFT.h5", "w") as f:
    f.create_group("data")
    f['data'].create_dataset('gene', data=gene)
    f['data'].create_dataset('snplist', data=snplist)
    f['data'].create_dataset('pindex', data=pindex)
    f['data'].create_dataset('Zscore_eQTLs_df', data=Zscore_eQTLs_df)
    f['data'].create_dataset('Zscore_GWAS_df', data=Zscore_GWAS_df)
    f['data'].create_dataset('geneExpression_corr_df', data=expression_corr)
    f['data'].create_dataset('LD_eQTL', data=LD_eQTL)
    f['data'].create_dataset('LD_GWAS', data=LD_GWAS)


In [ ]:
aaa

In [ ]:
gwasPath = "/home/xutingfeng/GIFT/data/GWAS/DNAJC16_gift.tsv" 
eQTLsLocation = "/home/xutingfeng/GIFT/data/eQTL/Kidney_Cortex/genes"
plink_vcor2_LDPath = "/home/xutingfeng/GIFT/data/pgen/DNAJC16_LD.unphased.vcor2"

geneExpressionPath = "/home/xutingfeng/GIFT/data/expression/gene_tpm_2017-06-05_v8_kidney_cortex.gct.gz"
Ensemble_gff3 =  "Homo_sapiens.GRCh38.111.chromosome.1.gff3.gz"




In [258]:
def detect_in_region(value, region_list):
    """
    Check if a value falls within any of the regions in the given region list.

    Parameters:
    value (float): The value to be checked.
    region_list (list): A list of tuples representing the regions. Each tuple should contain two elements: the lower bound and the upper bound of the region.

    Returns:
    bool: True if the value falls within any of the regions, False otherwise.
    """
    if isinstance(region_list[0], int):
        region_list = [region_list]
    for region in region_list:
        if all((value >= region[0], value < region[1])):

            return True 

    return False


def extract_cis_snps(df, pos_col,one_based_start_end):
    selector = df[pos_col].apply(lambda x: detect_in_region(x, one_based_start_end))
    return df[selector]

In [ ]:
from functools import reduce
# snplist is all genes snp concat together, so may have duplicate snp
def load_GTEx_gene_expression(geneExpression):
    geneExpression_df = pd.read_csv(geneExpression, sep = '\t', skiprows=2 ).iloc[:, 2:].set_index('Description').T
    return geneExpression_df

def preprocess(GWASPath, eQTLLocation, LDPath, geneExpressionPath, used_genes = None, gff=None, cis_range = 100,
    eQTLs_col_dict=None, gwas_col_dict=None,
    force_same = False
    ):
    """
    cis_range: int, default 100, means 100kb, the range to define cis eQTLs
    """

    if cis_range: 
        cis_range = cis_range*1000

    # Step1 load data 
    # read GWAS file
    if gwas_col_dict is None:
        gwas_col_dict = {}
    gwasPOS_col = gwas_col_dict.get("POS", "pos")
    gwasSNP_col = gwas_col_dict.get("SNP", "ID")
    gwasBETA_col = gwas_col_dict.get("BETA", "BETA")
    gwasSE_col = gwas_col_dict.get("SE", "SE")
    gwas = pd.read_csv(GWASPath, sep = "\t")[[gwasPOS_col, gwasSNP_col, gwasBETA_col, gwasSE_col]]

    # read eQTLs

    if eQTLs_col_dict is None:
        eQTLs_col_dict = {} 
    
    eQTLPOS_col = eQTLs_col_dict.get("POS", "BP_hg38")
    eQTLSNP_col = eQTLs_col_dict.get("SNP", "SNP")
    eQTLBETA_col = eQTLs_col_dict.get("BETA", "BETA")
    eQTLSE_col = eQTLs_col_dict.get("SE", "SE")
    eQTLs_list  = {i.stem:pd.read_csv( str(i), sep='\t')[[eQTLPOS_col, eQTLSNP_col, eQTLBETA_col, eQTLSE_col]] for i in Path(eQTLsLocation).glob("*") }

    # read LD 
    # TODO: support for different LD of eQTL and GWAS
    LD_martix = pd.read_csv(LDPath, sep = "\t", header = None)
    LD_variants = pd.read_csv(LDPath + ".vars", sep = "\t", header = None)
    LD_martix.columns = LD_variants[0].tolist()
    LD_martix.index = LD_variants[0].tolist()
    # read gene expression
    geneExpression_df = load_GTEx_gene_expression(geneExpressionPath)

    genes_in_analysis = list(eQTLs_list.keys())
            
    # check with geneExpression_df 
    genes_in_analysis = list(set(genes_in_analysis) & set(geneExpression_df.columns.tolist()))
    
    if used_genes:
        intersection = list(set(used_genes) & set(genes_in_analysis))
        if len(intersection) != len(used_genes):
            print("Some genes are not found in the eQTLs or gene expression data")
            print(f"final used {len(intersection)} genes while raw used {len(genes_in_analysis)} genes")
        genes_in_analysis = intersection

    if gff:
        gff = pyranges.read_gff3(gff)
        gene_gff = gff[gff.Feature == 'gene'].as_df()
        gene_in_gff = list(set(gene_gff.Name.tolist()) & set(genes_in_analysis))
        if len(gene_in_gff) != len(genes_in_analysis):
            print("Some genes are not found in the gff file", list(set(genes_in_analysis) - set(gene_in_gff)))
            print(f"final used {len(gene_in_gff)} genes while raw used {len(genes_in_analysis)} genes")
        genes_in_analysis = gene_in_gff

        # filter eQTL_list with cis snp 
        gff_filter_cis_eQTLs_list = {}
        gff_filter_cis_eQTLs_gene_list = [] 
        for gene in sorted(genes_in_analysis):
            gene_region = gene_gff[gene_gff.Name == gene].iloc[0][['Start', "End"]].tolist()
            upsteam = (gene_region[0] - cis_range, gene_region[0])
            downstream = (gene_region[1], gene_region[1] + cis_range)
            gene_cis_range = [upsteam, downstream]

            filted_cis_eQTLs_snp_df = extract_cis_snps(eQTLs_list[gene], eQTLPOS_col, gene_region, gene_cis_range)
            if filted_cis_eQTLs_snp_df.shape[0] > 0:
                gff_filter_cis_eQTLs_list[gene] = filted_cis_eQTLs_snp_df
                gff_filter_cis_eQTLs_gene_list.append(gene)
            print(f"gene {gene} has {filted_cis_eQTLs_snp_df.shape[0]} cis snps while raw has {eQTLs_list[gene].shape[0]} snps with cis range {cis_range//1000}kb and gene region is {gene_region} so the cis range of gene is {gene_cis_range}")

        genes_in_analysis = gff_filter_cis_eQTLs_gene_list
        eQTLs_list = gff_filter_cis_eQTLs_list
    

    geneExpression_df = geneExpression_df[genes_in_analysis]
    eQTLs_list = {gene: eQTLs_list[gene] for gene in genes_in_analysis if gene in eQTLs_list.keys()}

    # step2, generate the snplist
    if force_same:
        snplist = []

        for gene, gene_df in eQTLs_list.items():
            snplist.append(gene_df[eQTLSNP_col].tolist())
        from functools import reduce
        force_same_snplist = reduce(lambda x, y: list(set(x) & set(y)), snplist)

        if len(force_same_snplist) == 0:
            raise ValueError("No snps are shared by all genes")
        else:
            snplist =reduce(lambda x, y: x+ y,  [force_same_snplist] * len(snplist))
        print(len(force_same_snplist))
        force_same_eQTLs_list = {} 
        for gene, gene_df in eQTLs_list.items():
            force_same_eQTLs_list[gene] = gene_df[gene_df[eQTLSNP_col].isin(snplist)]

        eQTLs_list = force_same_eQTLs_list 

    else:
        snplist = []

        for gene, gene_df in eQTLs_list.items():
            snplist += gene_df[eQTLSNP_col].tolist()


    # step3 generate Zscore eQTLs
    snplist_df = pd.DataFrame({eQTLSNP_col: snplist})
    pindex = []
    Zscore_eQTLs_list = []
    for gene, gene_df in eQTLs_list.items():

        gene_df[f'TSTAT_{gene}'] = gene_df[eQTLBETA_col]/gene_df[eQTLSE_col]# TODO: accept pre calculated tstat
        if any(gene_df[f'TSTAT_{gene}'].isna()):
            print(gene_df[gene_df[f'TSTAT_{gene}'].isna()])
            raise ValueError(f"eQTLs of {gene} has NA value in your eQTLs data at TSTAT col, plz check")
        # print([eQTLSNP_col, 'TSTAT'])

        Zscore_eQTLs_df = snplist_df.merge(gene_df[[eQTLSNP_col, f'TSTAT_{gene}']], on = eQTLSNP_col, how = 'left')

        pindex.append(gene_df.shape[0])
        
        Zscore_eQTLs_list.append(Zscore_eQTLs_df.set_index(eQTLSNP_col))
    # Zscore_eQTLs =  np.stack(Zscore_eQTLs)
    assert all_equal == all((x.index == Zscore_eQTLs_list[0].index ).all() for x in Zscore_eQTLs_list)  # check if all index are the same
    Zscore_eQTLs_df = pd.concat(Zscore_eQTLs_list, axis = 1)

    # print(len(snplist))
    # step4 generate Zscore_GWAS     
    gwas['TSTAT'] = gwas[gwasBETA_col]/gwas[gwasSE_col]
    if any(gwas['TSTAT'].isna()):
        print(gwas[gwas['TSTAT'].isna()])
        raise ValueError(f"GWAS has NA value in your GWAS data at TSTAT col, plz check")
    snplist_df = pd.DataFrame({gwasSNP_col: snplist})
    Zscore_GWAS_df = snplist_df.merge(gwas[[gwasSNP_col, 'TSTAT']], on = gwasSNP_col, how = 'left').set_index(gwasSNP_col)
    # print(Zscore_GWAS_df.shape[0])
    # step5 match LD with snplist
    LD_eQTL = LD_martix.loc[snplist, snplist] # todo: support for different LD of eQTL and GWAS
    LD_GWAS = LD_martix.loc[snplist, snplist]


    #step6 geneExpression corr  
    geneExpression_corr_df = geneExpression_df.corr()
    # check all 
    print(Zscore_GWAS_df.shape[0], sum(pindex))
    assert sum(pindex) == Zscore_GWAS_df.shape[0], print(f"eQTLs snps are not equal to GWAS snps, eQTLs snps are {sum(pindex)} and GWAS snps are {Zscore_GWAS_df.shape[0]}")
    assert [col.split("_")[1] for col in Zscore_eQTLs_df.columns] == genes_in_analysis
    assert Zscore_GWAS_df.index.tolist() == Zscore_eQTLs_df.index.tolist()
    assert Zscore_GWAS_df.index.tolist() == snplist
    assert Zscore_eQTLs_df.index.tolist() == LD_eQTL.index.tolist()
    assert Zscore_GWAS_df.index.tolist() == LD_GWAS.index.tolist()
    assert geneExpression_corr_df.index.tolist() == genes_in_analysis
    print(f"eQTLs matrix shape is :{Zscore_eQTLs_df.shape}")
    return genes_in_analysis, snplist, pindex, Zscore_eQTLs_df, Zscore_GWAS_df, geneExpression_corr_df, LD_eQTL, LD_GWAS

In [ ]:
used_genes=["AGMAT", "DNAJC16"]

In [ ]:
gene, snplist, pindex, Zscore_eQTLs_df, Zscore_GWAS_df, geneExpression_corr_df, LD_eQTL, LD_GWAS = preprocess(gwasPath, eQTLsLocation, plink_vcor2_LDPath, geneExpressionPath, used_genes=used_genes, gff = Ensemble_gff3, force_same=True)


In [ ]:
Zscore_eQTLs_df

In [ ]:
Zscore_eQTLs_df.fillna(0, inplace=True)
Zscore_GWAS_df.fillna(0, inplace=True)

In [ ]:
# to h5 
import h5py
with h5py.File("GIFT.h5", "w") as f:
    f.create_group("data")
    f['data'].create_dataset('gene', data=gene)
    f['data'].create_dataset('snplist', data=snplist)
    f['data'].create_dataset('pindex', data=pindex)
    f['data'].create_dataset('Zscore_eQTLs_df', data=Zscore_eQTLs_df)
    f['data'].create_dataset('Zscore_GWAS_df', data=Zscore_GWAS_df)
    f['data'].create_dataset('geneExpression_corr_df', data=geneExpression_corr_df)
    f['data'].create_dataset('LD_eQTL', data=LD_eQTL)
    f['data'].create_dataset('LD_GWAS', data=LD_GWAS)




In [ ]:
# # to h5 
# import h5py
# with h5py.File("GIFT.h5", "w") as f:
#     f.create_group("data")
#     f['data'].create_dataset('gene', data=gene)
#     f['data'].create_dataset('snplist', data=snplist)
#     f['data'].create_dataset('pindex', data=pindex)
#     f['data'].create_dataset('Zscore_eQTLs', data=Zscore_eQTLs_df.values)
#     f['data'].create_dataset('Zscore_GWAS', data=Zscore_GWAS_df.values)
#     f['data'].create_dataset('geneExpression_corr', data=geneExpression_corr_df.values)
#     f['data'].create_dataset('LD_eQTL', data=LD_eQTL.values)
#     f['data'].create_dataset('LD_GWAS', data=LD_GWAS.values)


In [ ]:
gene